<a href="https://colab.research.google.com/github/sriraj0926/Capstone/blob/main/Capstone_Code_IC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow numpy nltk pillow

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import os
import string
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, Add
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from nltk.translate.bleu_score import corpus_bleu
import nltk

nltk.download('punkt')

# Define paths
dataset_dir = '/content/gdrive/MyDrive/Capstone_datasets/flickr8k_unzipped'  # Change this to the path where the dataset is located
images_dir = os.path.join(dataset_dir, 'Images')
captions_file = os.path.join(dataset_dir, 'captions.txt')

# Load the captions into a dictionary
def load_captions(file):
    captions = {}
    with open(file, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            image_id, image_caption = tokens[0], tokens[1:]
            image_id = image_id.split('.')[0]
            image_caption = ' '.join(image_caption)
            if image_id not in captions:
                captions[image_id] = []
            captions[image_id].append(image_caption)
    return captions

captions = load_captions(captions_file)

# Clean the captions
def clean_captions(captions):
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in captions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            desc = desc.split()
            desc = [word.lower() for word in desc]
            desc = [w.translate(table) for w in desc]
            desc = [word for word in desc if len(word) > 1]
            desc = [word for word in desc if word.isalpha()]
            desc_list[i] = ' '.join(desc)

clean_captions(captions)

# Convert the captions to a vocabulary
def to_vocabulary(captions):
    all_captions = set()
    for key in captions.keys():
        [all_captions.update(d.split()) for d in captions[key]]
    return all_captions

vocabulary = to_vocabulary(captions)

# Save the captions to a file
def save_captions(captions, filename):
    lines = []
    for key, desc_list in captions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc)
    data = '\n'.join(lines)
    with open(filename, 'w') as f:
        f.write(data)

save_captions(captions, 'captions.txt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
